In [2]:
from sklearn.base import BaseEstimator, TransformerMixin

import m as a

from datetime import datetime
import pandas_datareader as web
import pandas as pd

# sklearn libraries
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle

from sklearn.svm import SVR

from sklearn.model_selection import (train_test_split,
                                     cross_validate,
                                     GridSearchCV
                                    )

from sklearn.metrics import (r2_score, 
                             mean_squared_error
                            )

In [8]:
class SupportVectorRegression():
    """
    This is an example of the support vector machine regression class.
    """
    
    ticker = "ES=F"
    test_size = 0.3
    random_state = 42
    
    def __init__(self, 
                 kernel:str = None,
                 degree:int = 3,
                 gamma:float = 1.0,
                 coef0:float = 0.0,
                 C:float = 1.0,
                 epsilon:float = 0.1,
                 verbose:bool = False,
                 max_iter:int = -1
                ):

        if (kernel is None): # set rbf to default
            self.kernel = "rbf" 
        else:
            self.kernel = kernel

        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.C = C
        self.epsilon = epsilon
        self.verbose = verbose
        self.max_iter = max_iter

        # model parameters set to None (I have to come up with them later)
        self.X = None
        self.y = None
        self.coef_ = None
        self.intercept_ = None

    def download_data(self):
        """
        The below function download the data into the class.
        """
        end = datetime.today()
        start = datetime(end.year-2, end.month, end.day)
        ticker = web.DataReader(SupportVectorRegression.ticker, "yahoo", start, end) #snp500 futures
        
        t = a.TechnicalIndicators(ticker["Close"], ticker["High"], ticker["Low"], ticker["Volume"])
        df = pd.DataFrame(t)
        
        return self.df

    
    
    def create_Xy(self, X:float, y:float):
        """
        The below function sets the dependent and independent variables.
        """
        
        X = np.array(
            t.bollinger_bands()[
                ["Bollinger Bands Lower", "Bollinger Bands Middle", "Bollinger Bands Upper"]
                                ]).reshape(-1,3)
        
        y = np.array(ticker["Close"]).reshape(len(ticker["Close"]),1) # or -1   


    def feature_scaling(self):
        """
        The following function scales the data. Step 2: Data pre-processing. 
        """
        scaler = MinMaxScaler(feature_range=(0, 1))
        
        X_train = scaler.fit_transform(X_train)
        y_train = scaler.fit_transform(y_train)
        
        
    def train_test_split(self, X:np.ndarray, y:np.ndarray):    
        """
        The following function splits the data into train and test set. Step 3: feature engineering. 
        """
        X = X.values.reshape(-1,1) # -1 unknown dimension and we want numpy to figure it out, 1 column and numpy figures out the number of rows
        y = y.values.reshape(-1,1)
        
        # reshape must be done before train test split so that X,y are passed in the format required (1 col and many rows)
        X_train, X_test, y_train, y_test = train_test_split(X, 
                                                            y, 
                                                            test_size = SupportVectorRegression.test_size, 
                                                            random_state = SupportVectorRegression.random_state)
        
        return X_train, X_test, y_train, y_test
    
    
    def fit(self, X_train:np.ndarray, y_train:np.ndarray):
        """
        The following function fits the model over the training data. Step 4: fitting the model. 
        """
        X_train, y_train = train_test_split()
        
        y_ = np.where(y <= 0, -1,1)
        n_samples, n_features = X.shape
        
        SupportVectorReg = SVR(kernel = self.kernel,
                               degree = self.degree,
                               gamma = self.gamma,
                               coef0 = self.coef0,
                               tol = 0.001,
                               C = self.C,
                               epsilon = self.epsilon,
                               verbose = False,
                               max_iter = -1 
                              )
        
        SupportVectorReg.fit(X_train, y_train)

    def predict(self, X_test:np.array):
        y_pred = SupportVectorReg.predict(X_test)
        return y_pred
        
    def evaluation(self):
        """
        The following function evaluates how well the model performs on the test data. Step 6: model evaluation. 
        """
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        return rmse

In [ ]:
SVR()

In [ ]:
end = datetime.today()
start = datetime(end.year-2, end.month, end.day)
ticker = web.DataReader("ES=F", "yahoo", start, end) #snp500 futures#     def varyC(self):

t = a.TechnicalIndicators(ticker["Close"], ticker["High"], ticker["Low"], ticker["Volume"])

t.set_technical_indicators()
t.rsi()
t.rate_change()
t.stochastic_oscillator()
t.WilliamsR()

t.macd()
t.ema()
t.wma()
t.moving_average()

t.bollinger_bands()
t.average_true_range()

t.on_balance_volume()
t.money_flow_index()

In [ ]:
np.array(ticker["Close"]).reshape(len(ticker["Close"]),1)

In [ ]:
# print("C: {}".format(fitted_svr_model.C))
#     print("Epsilon: {}".format(fitted_svr_model.epsilon))
    
#     sc_X = StandardScaler()
# sc_y = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# y_train = sc_y.fit_transform(y_train)

# X_test = sc_X.fit_transform(X_test)
# y_test = sc_y.fit_transform(y_test)

# np.dot

# """
#         Parameters:
#         - kernel:   string indicating kernel "linear", "poly", "rbf", default "rbf" see above
#                     transforms train data so that data that cannot be linearly separated 
#                     are now linearly separated in a higher number of dimension spaces
#         - degree:
#         - gamma:
#         - coef0:
#         - C:
#         - epsilon: The value defines a margin of tolerance where no penalty is given to errors. 
#                    The larger it is, the larger errors you admit in your solution.
#         - verbose: output messages, useful for debugging or understading how the training performs
#         - max_iter: -1 for no limit
#         """        

#     def param_list(self):
#         """
#         The following function returns the list of parameters and the respective values.
#         """
#         return {"C": self.C, 
#                 "kernel": self.kernel, 
#                 "gamma": self.gamma}

# if __name__ == "__main__":


#     def param_grid(self):
        
#         param_grid = {'C': [ 1, 10, 100, 1000,10000], 
#               'gamma': [1,0.1,0.01,0.001,0.0001],
#               'kernel': ['rbf']}
        
#         pass